In [6]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [8]:
import cv2
import numpy as np
import onnxruntime
from prettytable import PrettyTable
from dotenv import load_dotenv
import os

In [10]:
load_dotenv()
MODEL_PATH = os.getenv("FACENET_ONNX_MODEL")
IMAGES_BASE_DIR = os.getenv("IMAGES_BASE_DIR")

In [15]:
def full_path(img):
    return os.path.join(IMAGES_BASE_DIR, img)

image_files = os.listdir(IMAGES_BASE_DIR)
image_paths = [full_path(img) for img in image_files if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

# for path in image_lst:
#     print(path)

### Load the model

FaceNet PyTorch, in particular the one we imported from vggface2 has an input of RGB 160,160 and an embedding output of size 512

In [13]:
session = onnxruntime.InferenceSession(MODEL_PATH, providers=["CPUExecutionProvider"])

input_tensor_name = session.get_inputs()[0].name
output_tensor_name = session.get_outputs()[0].name


session.get_inputs()[0].shape, session.get_outputs()[0].shape

([1, 3, 160, 160], [1, 512])

### Helper functions
Ensure the input size is met, convert it from BGR to RGB and normalize the pixel values before passing it on to facenet model 

In [14]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (160, 160))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.transpose((2, 0, 1))
    image = np.expand_dims(image, axis=0)
    image = image.astype(np.float32)
    image /= 255.0
    return image

def normalize_embedding(embedding):
    return embedding / np.linalg.norm(embedding)

def get_face_embedding(image_path):
    image = preprocess_image(image_path)
    result = session.run([output_tensor_name], {input_tensor_name: image})[0]
    embedding = result[0]
    return normalize_embedding(embedding)

def cosine_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2)


In [16]:
embeddings = [get_face_embedding(path) for path in image_paths]
embedding_array = np.array(embeddings)

### Verify results
We use cosine similarity as a metric to evaluate our embedding performance

In [17]:
table = PrettyTable()
table.field_names = [""] + [path.split("/")[-1] for path in image_paths]
for i, embedding1 in enumerate(embeddings):
    row = [image_paths[i].split("/")[-1]]
    for embedding2 in embeddings:
        similarity = cosine_similarity(embedding1, embedding2)
        row.append(f"{similarity:.4f}")
    table.add_row(row)

print("Similarity Matrix:")
print(table)


Similarity Matrix:
+-------------------------+-------------------------+-------------------------+------------------------+-----------------------+-------------------+-------------------+-----------------------+------------------------+-----------------------+-------------------+
|                         | Aicha_El_Ouafi_0003.jpg | Aicha_El_Ouafi_0001.jpg | Aaron_Peirsol_0001.jpg | Frank_Solich_0001.jpg | Abdullah_0003.jpg | Abdullah_0004.jpg | Frank_Solich_0004.jpg | Aaron_Peirsol_0002.jpg | Frank_Solich_0002.jpg | Abdullah_0002.jpg |
+-------------------------+-------------------------+-------------------------+------------------------+-----------------------+-------------------+-------------------+-----------------------+------------------------+-----------------------+-------------------+
| Aicha_El_Ouafi_0003.jpg |          1.0000         |          0.8751         |        -0.0327         |         0.1167        |      -0.0617      |      -0.0064      |         0.1630        |   

### Check DBScan cluster results
Density based clustering algorithm, see scikit learn documentation/Wikipedia.

In [18]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.6, min_samples=2, metric='cosine')
cluster_labels = dbscan.fit_predict(embedding_array)
clusters = {}
for path, label in zip(image_paths, cluster_labels):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(path.split("/")[-1])

for cluster_id, image_names in clusters.items():
    if cluster_id == -1:
        print("outliers:")
    else:
        print(f"Cluster {cluster_id}:")
    for image_name in image_names:
        print(f"  {image_name}")
    print()

Cluster 0:
  Aicha_El_Ouafi_0003.jpg
  Aicha_El_Ouafi_0001.jpg

Cluster 1:
  Aaron_Peirsol_0001.jpg
  Aaron_Peirsol_0002.jpg

Cluster 2:
  Frank_Solich_0001.jpg
  Frank_Solich_0004.jpg
  Frank_Solich_0002.jpg

Cluster 3:
  Abdullah_0003.jpg
  Abdullah_0004.jpg
  Abdullah_0002.jpg

